<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-700)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['ACC', 'ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANITRANS',
       'AMBUJACEM', 'APOLLOHOSP', 'ASIANPAINT', 'AUBANK', 'AUROPHARMA',
       'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE',
       'BANDHANBNK', 'BANKBARODA', 'BATAINDIA', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CANBK',
       'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DLF', 'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD',
       'FEDERALBNK', 'GAIL', 'GILLETTE', 'GLAND', 'GODREJCP', 'GODREJIND',
       'GRASIM', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDPETRO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL',
       'INDIANB', 'INDIGO', 'INDIGOPNTS', 'INDUSINDBK', 'INDUSTOWER',
       'INFY', 'IOC', 'IRCTC', 'ITC', 'JINDALSTEL', 'JSWSTEEL',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'LUPIN', 'M&M', 'MARICO',
       'MARUTI', '

In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

L    67
H    31
M    27
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    70
1    55
Name: StarStock, dtype: int64

In [7]:
# latest quarter
mypf['LatestQ'].value_counts()

0    74
1    51
Name: LatestQ, dtype: int64

In [8]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    110
1     15
Name: InPortfolio, dtype: int64

In [9]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [10]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [11]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQ'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQ'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [12]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-04-05 00:00:00+05:30,17557.0,^NSEI,0.33,0.26,-0.44,58.0,18812.0,15294.0,35.67,7.0
2023-04-13 00:00:00+05:30,17828.0,^NSEI,1.54,0.51,-0.13,66.0,18812.0,15294.0,27.97,6.0
2023-04-21 00:00:00+05:30,17624.0,^NSEI,0.11,0.60,-0.02,56.0,18812.0,15294.0,33.77,7.0
2023-04-28 00:00:00+05:30,18065.0,^NSEI,2.32,0.56,0.02,71.0,18812.0,15294.0,21.23,4.0
2023-05-08 00:00:00+05:30,18264.0,^NSEI,3.15,0.58,0.13,68.0,18812.0,15294.0,15.58,3.0


In [13]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [14]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [15]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50',	'RSI',	'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQ']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 15


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQ
Stock,,,,,,,,,,,,,
INDIGOPNTS.NS,1275.0,-3.03,-0.91,1.03,82.0,82.77,110.0,1152.30,104.9,51.5,H,51.0,0
JUBLFOOD.NS,466.0,-11.74,-0.95,0.00,63.0,90.85,94.0,448.52,90.5,73.1,H,19.0,0
IRCTC.NS,628.0,-4.98,0.30,-0.17,66.0,69.97,85.0,618.80,77.4,53.3,H,31.0,0
INFY.NS,1266.0,-14.72,-0.74,-2.30,38.0,93.63,50.0,1252.12,29.0,21.9,H,24.0,1
BATAINDIA.NS,1502.0,-9.77,-0.66,0.14,64.0,85.37,45.0,1495.89,126.0,60.3,H,52.0,0
DMART.NS,3678.0,-6.20,-0.26,0.20,72.0,76.63,45.0,3501.37,160.1,102.0,H,36.0,0
PAGEIND.NS,41372.0,-6.71,-0.20,0.60,69.0,49.85,30.0,38201.00,97.7,67.5,H,31.0,0
GILLETTE.NS,4596.0,-6.82,-0.55,-0.56,64.0,76.68,29.0,4353.00,61.2,50.3,M,18.0,0
BAJFINANCE.NS,6658.0,1.15,0.36,-0.08,76.0,47.15,19.0,6044.41,52.6,35.0,H,33.0,1


In [16]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [17]:
# all time high sales and profit TTM

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ADANIENT.NS,1888.0,-35.32,-0.69,0.79,53.0,4165.0,1194.0,76.64,121.0,1.0,0.0,M,1
LTIM.NS,4518.0,-1.08,0.46,-0.54,53.0,7439.0,3761.0,79.42,65.0,1.0,0.0,M,1
ICICIGI.NS,1083.0,-7.21,-0.60,-0.46,50.0,1620.0,1060.0,95.89,50.0,1.0,0.0,M,1
BANDHANBNK.NS,235.0,-4.92,-0.81,0.00,68.0,346.0,187.0,69.81,47.0,1.0,1.0,L,0
TATAPOWER.NS,203.0,-6.16,0.00,0.00,58.0,287.0,111.0,47.73,41.0,1.0,0.0,M,1
SBICARD.NS,815.0,-0.50,-0.12,0.53,72.0,1133.0,673.0,69.13,39.0,1.0,0.0,M,1
BERGEPAINT.NS,620.0,2.78,0.17,1.02,68.0,858.0,540.0,74.84,38.0,1.0,0.0,M,0
CIPLA.NS,933.0,-9.30,0.00,-1.11,60.0,1173.0,856.0,75.71,26.0,1.0,0.0,M,0
TCS.NS,3251.0,1.17,0.12,-0.90,57.0,3886.0,2915.0,65.40,20.0,1.0,0.0,M,1


In [18]:
# all time high sales and profit TTM

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 17


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ITC.NS,431.0,25.06,1.74,1.27,81.0,431.0,189.0,0.00,0.0,1.0,0.0,M,0
TITAN.NS,2751.0,8.79,1.23,0.72,77.0,2770.0,1662.0,1.71,1.0,1.0,1.0,H,1
LT.NS,2365.0,15.33,1.80,0.81,66.0,2384.0,1432.0,2.00,1.0,1.0,0.0,M,0
BRITANNIA.NS,4598.0,13.40,1.11,0.16,70.0,4626.0,2945.0,1.67,1.0,1.0,0.0,M,1
ICICIBANK.NS,938.0,6.94,1.14,0.69,75.0,953.0,619.0,4.49,2.0,1.0,1.0,H,1
AUBANK.NS,691.0,10.58,0.64,0.97,65.0,717.0,493.0,11.61,4.0,1.0,0.0,M,1
INDIANB.NS,315.0,25.12,3.17,1.72,54.0,334.0,112.0,8.56,6.0,1.0,1.0,H,1
SBIN.NS,584.0,4.78,0.90,0.74,69.0,626.0,400.0,18.58,7.0,1.0,0.0,M,0
BHARTIARTL.NS,794.0,2.55,0.65,0.13,61.0,851.0,520.0,17.22,7.0,1.0,1.0,H,0


In [19]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS','SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ACC.NS,1767.0,-19.40,-0.96,-0.46,53.0,2748.0,1614.0,86.51,56.0,0.0,0.0,L,0
AMBUJACEM.NS,406.0,-10.68,0.22,1.07,63.0,588.0,275.0,58.15,45.0,0.0,0.0,L,0
PGHH.NS,13832.0,-0.77,0.09,0.19,46.0,15778.0,12000.0,51.51,14.0,0.0,0.0,L,0
NESTLEIND.NS,21983.0,13.37,0.92,1.42,78.0,22020.0,15919.0,0.61,0.0,1.0,1.0,H,0
SIEMENS.NS,3690.0,23.90,1.54,1.23,81.0,3690.0,1922.0,0.00,0.0,1.0,1.0,H,0


In [20]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
INDUSTOWER.NS,151.0,-17.20,-1.65,-1.32,56.0,303.0,137.0,91.57,101.0,1.0,1.0,H,1
BANDHANBNK.NS,235.0,-4.92,-0.81,0.00,68.0,346.0,187.0,69.81,47.0,1.0,1.0,L,0


In [21]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
INDUSTOWER.NS,151.0,-17.20,-1.65,-1.32,56.0,303.0,137.0,91.57,101.0,1.0,1.0,H,1
HDFCLIFE.NS,550.0,1.39,-0.18,0.59,61.0,757.0,461.0,69.93,38.0,1.0,1.0,H,1
BANDHANBNK.NS,235.0,-4.92,-0.81,0.00,68.0,346.0,187.0,69.81,47.0,1.0,1.0,L,0


In [22]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 7


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
SIEMENS.NS,3690.0,23.90,1.54,1.23,81.0,3690.0,1922.0,0.00,0.0,1.0,1.0,H,0
NESTLEIND.NS,21983.0,13.37,0.92,1.42,78.0,22020.0,15919.0,0.61,0.0,1.0,1.0,H,0
TITAN.NS,2751.0,8.79,1.23,0.72,77.0,2770.0,1662.0,1.71,1.0,1.0,1.0,H,1
ICICIBANK.NS,938.0,6.94,1.14,0.69,75.0,953.0,619.0,4.49,2.0,1.0,1.0,H,1
BANKBARODA.NS,183.0,17.31,2.56,1.18,61.0,193.0,71.0,8.20,5.0,1.0,1.0,H,0
INDIANB.NS,315.0,25.12,3.17,1.72,54.0,334.0,112.0,8.56,6.0,1.0,1.0,H,1
CANBK.NS,313.0,12.26,1.79,0.68,60.0,337.0,139.0,12.12,8.0,1.0,1.0,H,1


In [23]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
NESTLEIND.NS,21983.0,13.37,0.92,1.42,78.0,22020.0,15919.0,0.61,0.0,1.0,1.0,H,0
DRREDDY.NS,4907.0,11.35,0.59,0.91,64.0,5451.0,3669.0,30.53,11.0,1.0,1.0,H,0


In [24]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 3


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
HDFCLIFE.NS,550.0,1.39,-0.18,0.59,61.0,757.0,461.0,69.93,38.0,1.0,1.0,H,1
BANDHANBNK.NS,235.0,-4.92,-0.81,0.00,68.0,346.0,187.0,69.81,47.0,1.0,1.0,L,0
INDUSTOWER.NS,151.0,-17.20,-1.65,-1.32,56.0,303.0,137.0,91.57,101.0,1.0,1.0,H,1


In [25]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 40


,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ADANITRANS.NS,949.0,-61.47,-3.17,-0.32,42.0,4105.0,643.0,91.16,333.0,1.0,-1.0,L,0
PEL.NS,748.0,-23.69,-4.69,-1.80,60.0,2866.0,635.0,94.94,283.0,-1.0,0.0,L,1
ADANIGREEN.NS,917.0,-45.95,-3.12,3.73,51.0,2970.0,462.0,81.86,224.0,-1.0,-1.0,L,1
GLAND.NS,1388.0,-20.81,-3.42,0.31,61.0,4316.0,1154.0,92.60,211.0,1.0,0.0,M,0
IBREALEST.NS,69.0,-5.14,1.37,1.69,62.0,191.0,47.0,84.72,177.0,-1.0,-1.0,L,0
ADANIENT.NS,1888.0,-35.32,-0.69,0.79,53.0,4165.0,1194.0,76.64,121.0,1.0,0.0,M,1
NETWORK18.NS,57.0,-10.78,-1.56,0.00,51.0,114.0,45.0,82.61,100.0,-1.0,-1.0,L,1
ZEEL.NS,191.0,-19.24,-0.42,-0.99,39.0,367.0,164.0,86.70,92.0,-1.0,1.0,L,0
WIPRO.NS,380.0,-3.95,-0.51,-0.80,54.0,713.0,356.0,93.28,88.0,-1.0,0.0,L,1


In [26]:
stock_id = 'CHOLAFIN.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
CHOLAFIN.NS,1002.0,33.23,1.73,2.0,87.0,1002.0,473.0,0.0,0.0,NaN,NaN,L,1
